<a href="https://colab.research.google.com/github/Elma-dev/Hands_on_Tokenizer/blob/main/build_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
word="slm from me i'm abdeljalil and i will buold and train tokenizer from scratch like bpe 🅤🅝🅘🅒🅞🅓🅔"
# c -> ascii
tokens=list(word.encode("utf-8")) # represent word as a list of byte
print(tokens)
print(f"len word:{len(word)} , len tokens {len(tokens)}")
# token means each char has an ascii code
print(f"ascii('s') -> {ord(word[0])}")

[115, 108, 109, 32, 102, 114, 111, 109, 32, 109, 101, 32, 105, 39, 109, 32, 97, 98, 100, 101, 108, 106, 97, 108, 105, 108, 32, 97, 110, 100, 32, 105, 32, 119, 105, 108, 108, 32, 98, 117, 111, 108, 100, 32, 97, 110, 100, 32, 116, 114, 97, 105, 110, 32, 116, 111, 107, 101, 110, 105, 122, 101, 114, 32, 102, 114, 111, 109, 32, 115, 99, 114, 97, 116, 99, 104, 32, 108, 105, 107, 101, 32, 98, 112, 101, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148]
len word:93 , len tokens 114
ascii('s') -> 115


In [ ]:
def pair_counters(encode_idxs):
  pair_count={}
  for i in range(len(encode_idxs)-1):
    pair=(encode_idxs[i],encode_idxs[i+1])
    pair_count[pair]=pair_count.get(pair,0)+1
  return pair_count
# test
pairs_count=pair_counters(tokens)
pairs_count

{(115, 108): 1,
 (108, 109): 1,
 (109, 32): 4,
 (32, 102): 2,
 (102, 114): 2,
 (114, 111): 2,
 (111, 109): 2,
 (32, 109): 1,
 (109, 101): 1,
 (101, 32): 3,
 (32, 105): 2,
 (105, 39): 1,
 (39, 109): 1,
 (32, 97): 3,
 (97, 98): 1,
 (98, 100): 1,
 (100, 101): 1,
 (101, 108): 1,
 (108, 106): 1,
 (106, 97): 1,
 (97, 108): 1,
 (108, 105): 2,
 (105, 108): 2,
 (108, 32): 2,
 (97, 110): 2,
 (110, 100): 2,
 (100, 32): 3,
 (105, 32): 1,
 (32, 119): 1,
 (119, 105): 1,
 (108, 108): 1,
 (32, 98): 2,
 (98, 117): 1,
 (117, 111): 1,
 (111, 108): 1,
 (108, 100): 1,
 (32, 116): 2,
 (116, 114): 1,
 (114, 97): 2,
 (97, 105): 1,
 (105, 110): 1,
 (110, 32): 1,
 (116, 111): 1,
 (111, 107): 1,
 (107, 101): 2,
 (101, 110): 1,
 (110, 105): 1,
 (105, 122): 1,
 (122, 101): 1,
 (101, 114): 1,
 (114, 32): 1,
 (32, 115): 1,
 (115, 99): 1,
 (99, 114): 1,
 (97, 116): 1,
 (116, 99): 1,
 (99, 104): 1,
 (104, 32): 1,
 (32, 108): 1,
 (105, 107): 1,
 (98, 112): 1,
 (112, 101): 1,
 (32, 240): 1,
 (240, 159): 7,
 (159, 133): 

In [ ]:
# get common redandent pair
max(pairs_count,key=pairs_count.get)

(240, 159)

In [ ]:
# this function is to represent pair number as another number (1,2) -> 55
# that's will reduce the size of encoder list (compress)
def merge(encode_idxs,pair,new_id):
  new_encode_idxs=[]
  i=0
  while i<len(encode_idxs):
    if i!=len(encode_idxs)-1 and (encode_idxs[i],encode_idxs[i+1])==pair:
      new_encode_idxs.append(new_id)
      i+=2
    else:
      new_encode_idxs.append(encode_idxs[i])
      i+=1
  return new_encode_idxs
# test
print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))
# -----------
tokens2=merge(tokens,(240, 159),256)
print(f"""old size of tokenizer {len(tokens)}, new size {len(tokens2)} reduce by {len(tokens)-len(tokens2)} """)

[5, 6, 99, 9, 1]
old size of tokenizer 114, new size 107 reduce by 7 


In [ ]:
# do merge n times
n=10
new_tokens=tokens
merges={}
for i in range(n):
  pair_count=pair_counters(new_tokens)
  top_pair=max(pair_count,key=pair_count.get)
  new_tokens=merge(new_tokens,top_pair,256+i)
  merges[top_pair]=256+i
  print(f"{top_pair} -> {256+i}")

(240, 159) -> 256
(256, 133) -> 257
(109, 32) -> 258
(101, 32) -> 259
(100, 32) -> 260
(102, 114) -> 261
(261, 111) -> 262
(262, 258) -> 263
(108, 105) -> 264
(108, 32) -> 265


In [ ]:
print(f"""old size of tokenizer {len(tokens)}, new size {len(new_tokens)} reduce by {len(tokens)-len(new_tokens)} """)

old size of tokenizer 114, new size 80 reduce by 34 


In [ ]:
vocab={i:bytes([i]) for i in range(256)}
for (p0,p1),i in merges.items():
  vocab[i]=vocab[p0]+vocab[p1]

def decode(ids):
  tokens=b"".join([vocab[i] for i in ids])
  text=tokens.decode("utf-8",errors="replace")
  return text
decode([128])

'�'

In [ ]:
def encode(text):
  tokens=list(text.encode("utf-8"))
  while len(tokens)>2:
    pair_count=pair_counters(tokens)
    pair=min(pair_count,key=lambda p: merges.get(p,float("inf")))
    if pair not in merges:
      break
    idx=merges[pair]
    tokens=merge(tokens,pair,idx)
  return tokens

In [ ]:
# test
text="hello world"
decode(encode(text))

'hello world'

In [ ]:
import regex as re
class Tokenizer():
  def __init__(self,re_pattern):
    self.merges={}
    self.last_id=255
    self.vocab={i:bytes([i]) for i in range(256)}
    self.compiled_pattern=re.compile(re_pattern)

  def train(self,text,nbr_tokens):
    chunks=re.findall(self.compiled_pattern,text)
    tokens_ids=[list(map(int,chunk.encode("utf-8")))  for chunk in chunks]
    for i in range(nbr_tokens):
      pair_count={}
      for token_ids in tokens_ids:
        self.__pair_counter(token_ids,pair_count)
      pair=max(pair_count,key=pair_count.get)
      tokens_ids=self.__merge(tokens_ids,pair,self.last_id+1)
      self.merges[pair]=self.last_id+1
      self.vocab[self.last_id+1]=self.vocab[pair[0]]+self.vocab[pair[1]]
      self.last_id+=1
    return tokens_ids

  def __pair_counter(self,tokens_ids,pair_count):
    for i in range(len(tokens_ids)-1):
      pair=(tokens_ids[i],tokens_ids[i+1])
      pair_count[pair]=pair_count.get(pair,0)+1
    return pair_count

  def __merge(self,tokens_ids,pair,new_id):
    new_encode_idxs=[]
    i=0
    while i<len(tokens_ids):
      if i!=len(tokens_ids)-1 and (tokens_ids[i],tokens_ids[i+1])==pair:
        new_encode_idxs.append(new_id)
        i+=2
      else:
        new_encode_idxs.append(tokens_ids[i])
        i+=1
    return new_encode_idxs

  def encode(self,text):
    chunks=re.findall(self.compiled_pattern,text)
    results=[]
    for text in chunks:
      tokens=list(text.encode("utf-8"))
      while len(tokens)>2:
        pair_count={}
        self.__pair_counter(tokens,pair_count)
        pair=min(pair_count,key=lambda p: self.merges.get(p,float("inf")))
        if pair not in self.merges:
          break
        idx=self.merges[pair]
        tokens=self.__merge(tokens,pair,idx)
      results.extend(tokens)
    return results

  def decode(self,ids):
    tokens=b"".join([self.vocab[i] for i in ids])
    text=tokens.decode("utf-8",errors="replace")
    return text


In [ ]:
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes thousands of emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters."
tokenizer=Tokenizer(DARIJA_SPLIT_PATTERN)

In [ ]:
tokenizer.train(valtext,30)
"nti hya lmerda"==tokenizer.decode(tokenizer.encode("nti hya lmerda"))

ValueError: max() arg is an empty sequence

In [ ]:
tokenizer.encode("nti hya lmerda")

[110, 116, 105, 32, 104, 121, 97, 32, 108, 109, 101, 114, 100, 97]

In [ ]:
DARIJA_SPLIT_PATTERN = r"""
    \'(?:[لب]|ه[او]|هم|ها|ك|ني)|
    ?[\p{Arabic}\p{Latin}]+|
    ?[\p{N}٠-٩]+|
    ?[^\s\p{Arabic}\p{Latin}\p{N}٠-٩]+|
    \s+(?!\S)|
    \s+ # Whitespace
"""